## Fabric Scanner API
This notebook calls the GetModifiedWorkspaces, PostWorkspaceInfo, GetScanStatus and GetScanResults APIs (collectively known as the [Scanner APIs](https://learn.microsoft.com/en-us/power-bi/enterprise/service-admin-metadata-scanning)) and some other admin APIs to create the following Delta tables in a Lakehouse:
- **Capacities**: A list of all Premium/Fabric/Embedded capacities in a tenant
- **Workspaces**: A list of all workspaces in the Fabric tenant
- **WorkspacePermissions**: A list of all individuals/groups with permissions to the workspace, their role and their Graph API identifier
- **Reports**: A list of all individuals/groups with permissions to the workspace
- **SemanticModelPermissions**: A list of all individuals/groups with permissions to the semantic model, their role and their Graph API identifier
- **PowerBILineage**: A calculated cartesian product fact table that - when used in conjunction with measures and relationships in a Semantic Model with these tables - can replicate the lineage view for Power BI workloads in the Fabric service but across an entire tenant

**Important Notes:**
- These tables are not slowly changing dimensions - they represent the current deployed artifacts in your Fabric tenant. Workspaces, Capacities, Domains, Apps, Artifacts, etc that are deleted may not appear
- Set _fullScan_ parameter to True to performan an initial scan of all workspaces in the tenant. Setting _fullScan_ to False performs a scan of all workspaces modified since that initial scan
    - When running this with _fullScan_ parameter to True, expect it to take a long time, especially in large tenants (>50K workspaces) or if you have all scan options set to "true"
    - My tenant with ~80K workspaces took approximately 3 hours for an initial scan
    - The PostWorkspaceInfo API has 429 limitations (500 requests per hour and only 16 active requests per hour) and that is the primary cause for the extended run time
    - This Notebook was written primarily to avoid long running and complex Spark operations where possible so the Scanner API calls are written in basic Python. So it's executing on the driver only but it gets the job done given the constraints. If you can write better PySpark than I can please, please contribute and let me know how I can help!
    - Subsequent runs of this notebook will run faster, so schedule frequent incremental scans
- Set _datasetSchema_ parameter to "false" to speed up generation and retrieval of workspace scans as the volume of data is considerably smaller
- Set _powerBIAPIBaseUri_ parameter to your tenant's cloud location (i.e "https://app.powerbi.com/v1.0/myorg" for Commercial, "https://app.powerbigov.us/v1.0/myorg" for GCC, etc)
- Your tenant may have additional datasources than the ones covered by the case() operation - in that case execute the query to find all datasources that "Require Mapping" and modify 
the case statement

**Known Issues:**
- These APIs are useful but not perfect:
    - If a semantic model or dataflow uses the Power Platform Dataflows connector (not the Legacy Dataflows connector) datasource lineage is not available from the scanner API.
    - The APIs do not return datasources for Paginated Reports
    - The GetAppsAsAdmin APIs do not currently include the multiple audiences feature
    - The Scanner APIs do not contain information about semantic model/dataflow size 

**Prerequisites:**
- A Power BI Admin must turn on the "[Allow Service Principals to use read-only Admin APIs](https://learn.microsoft.com/en-us/power-bi/enterprise/read-only-apis-service-principal-authentication)" feature. 
- If the Allow Service Principals to use read-only Admin APIs is turned on _and limited to specific security groups_, the service principal used to acquire the bearer token must be in one of the group(s) allowed to use the read-only Admin APIs
- The service principal used to acquire the bearer token must have Tenant.Read.All or Tenant.ReadWrite.All permissions
- **This notebook assumes you are storing service principal Client ID and Client Secret in an Azure Key Vault**. Fabric will retrieve them securely, use them during Notebook runs as obscure the values in logs and cell outputs. Avoid hard-coding service principal secrets in Notebooks

**Future Work:**
- Direct Lake Semantic model .pbip file

### Import libraries and set Spark session settings

In [68]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from delta.tables import *
import requests 
import json
import datetime
import time

spark.conf.set('spark.databricks.delta.schema.autoMerge.enabled','true')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 70, Finished, Available)

### Define Parmaters for Key Vault, PostWorkspaceInfo API and Lakehouse File directories

In [69]:
# Name of the Key Vault
nameOfKeyVault = 'powerbi-admin-keyvault' # Name of the Key Vault

# Names of the secrets saved in Key Vault 
tenantId_SecretName = 'TenantID'   # Tenant ID secret name
clientId_SecretName = 'powerbi-admin-sp-clientid'   # Name for Client ID of Service Principal
clientSecret_SecretName = 'powerbi-admin-sp-clientsecret' # Name for Client Secret of Service Principal

# Base URLS for Power BI and Key Vault
powerBIAPIBaseUri = 'https://api.powerbi.com/v1.0/myorg/'
fabricAPIBaseUri = 'https://api.fabric.microsoft.com/v1/'

# Scan options
fullScan = False
lineage = 'true'
datasourceDetails = 'true'
datasetSchema = 'true'
datasetExpressions = 'true'
getArtifactUsers = 'true'

# folder/file names
nameOfTempParquetFileFolder = 'Files/ScannerAPI/TempParquet/'
nameOfWorkspacesFileFolder = 'Files/ScannerAPI/ScanResults/Workspaces/'
nameOfDatasourcesFileFolder = 'Files/ScannerAPI/ScanResults/Datasources/'
nameOfControlFileFolder = 'Files/ScannerAPI/Reference'

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 71, Finished, Available)

In [70]:
def fxGetFabricBearerToken():
    keyvault = f'https://{nameOfKeyVault}.vault.azure.net/'
    # Begin don't hard code this stuff
    tenant_id = mssparkutils.credentials.getSecret(keyvault,tenantId_SecretName)
    client_id = mssparkutils.credentials.getSecret(keyvault,clientId_SecretName)
    client_secret = mssparkutils.credentials.getSecret(keyvault,clientSecret_SecretName)
    # End don't hard code this stuff
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/token'
    data = f'grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}&resource=https://analysis.windows.net/powerbi/api'  
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    response = requests.post(url, headers=headers, data=data)
    return response.json()["access_token"]

def fxRequestWithRetry(method, url, data, num_retries=3, success_list=[200, 202, 404], **kwargs):
    bearer_token = fxGetFabricBearerToken()
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {bearer_token}'}
    for i in range(num_retries):
        try:
            response = requests.request(method, url, headers=headers, data=data, **kwargs)
            if response.status_code in success_list:
                ## Return response if successful
                return response
            ## Captures the 500 requests in an hour limit
            if response.status_code == 429 and response.headers.get('Retry-After',None) is not None:
                waitTime = int(response.headers['Retry-After'])
                time.sleep(waitTime)
            ## Captures the 16 simultaneous requests limit
            if response.status_code == 429 and response.headers.get('Retry-After',None) is None:
                waitTime = 120
                time.sleep(waitTime)
        except requests.exceptions.ConnectionError:
            pass
    return None

def fxMergeDeltaTable(tableName,operationType,key):
    if DeltaTable.isDeltaTable(spark, "Tables/" + tableName):
        toUpdatePath="Tables/" + tableName + "/"
        sourcePath=nameOfTempParquetFileFolder + tableName 
        sdfToUpdate = DeltaTable.forPath(spark,toUpdatePath)
        sdfSource= spark.read.format("parquet").load(sourcePath)

        if operationType == 'Merge Upsert':
            (sdfToUpdate.alias("t")
            .merge(sdfSource.alias("s"), key)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
            )
        if operationType == 'Merge Delete':
            (sdfToUpdate.alias("t")
            .merge(sdfSource.alias("s"), key)
            .whenMatchedDelete()
            .whenNotMatchedInsertAll()
            .execute()
            )
    else: 
            sourcePath=nameOfTempParquetFileFolder + tableName 
            sdf = spark.read.format("parquet").load(sourcePath)
            sdf.write.mode("overwrite").option("mergeSchema","true").format("delta").save("Tables/" + tableName)
    return f'{tableName} is now up to date'     


StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 72, Finished, Available)

In [71]:
# set an anchor date for the forthcoming REST API calls
modifiedSinceNext = str(datetime.datetime.utcnow().isoformat()) + '0Z'

# For full scans, the cell creates an artificial JSON object, checks if all the directories required to run the notebook exist and if not, creates them
# The cell then writes the artificial JSON object to the Control.json file and creates the URL required for the forthcoming REST API calls
# For incremental scans, the cell opens the Control.json file, finds the modifiedSince attribute value and composes the URL required for the forthcoming REST API calls
if fullScan == True:
    j = {'modifiedSince':'1999-01-01T00:00:00.0000000Z'}
    mssparkutils.fs.mkdirs(f'{nameOfControlFileFolder}')
    mssparkutils.fs.mkdirs(f'{nameOfDatasourcesFileFolder}')
    mssparkutils.fs.mkdirs(f'{nameOfWorkspacesFileFolder}')
    with open(f'/lakehouse/default/{nameOfControlFileFolder}/Control.json', 'w') as f:
        f.write(json.dumps(j))
        f.close()
    url = f'{powerBIAPIBaseUri}/admin/workspaces/modified'
    modifiedSince = modifiedSinceNext
else:
    with open(f'/lakehouse/default/{nameOfControlFileFolder}/Control.json', 'r') as f:
        j = json.load(f)
    modifiedSince = j['modifiedSince']
    url = f'{powerBIAPIBaseUri}/admin/workspaces/modified?modifiedSince={modifiedSince}'

## call GetModifiedWorkspaces to return the list of workspaces based on the url generated above
response = fxRequestWithRetry("GET", url, '')
modifiedWorkspaces = response.json() 
workspaceCount = len(modifiedWorkspaces)
if fullScan == True:
    print(f'{str(workspaceCount)} total workspaces')
else: 
    print(f'{str(workspaceCount)} workspaces modified since {modifiedSince}')

## transform output of GetModifiedWorkspaces to format expected by PostWorkspaceInfo
workspaceIds = []
for modifiedWorkspace in modifiedWorkspaces:
    workspaceIds.append(modifiedWorkspace['id'])

## batch the workspaces based on batchSize. Maxium value for batchSize is 100
batchSize = 100
batches = [workspaceIds[i:i + batchSize] for i in range(0, len(workspaceIds), batchSize)]

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 73, Finished, Available)

5 workspaces modified since 2024-04-04T04:05:11.4956530Z


In [72]:
## batch Workspace IDs into batches of 100 since API has a payload limition, prepare payload, call PostWorkspaceInfo and create list of all scan IDs
scanCount = int(workspaceCount / 100) + (workspaceCount % 100 > 0)
print(f'Starting processing of {str(scanCount)} workspace batches')
batchNumber = 0
scans = []
for batch in batches:
    if batchNumber == scanCount:
            break
    else:
        batchNumber = batchNumber + 1
        payload = '{"workspaces": ' + str(json.dumps(batch)) + '}'
        print('Sending Scan ' + str(batchNumber) + ' to PostWorkspaceInfo API')
        url = f'{powerBIAPIBaseUri}/admin/workspaces/getInfo?lineage={lineage}&datasourceDetails={datasourceDetails}&datasetSchema={datasetSchema}&datasetExpressions={datasetExpressions}&getArtifactUsers={getArtifactUsers}'
        response = fxRequestWithRetry("POST",url,payload)
        scan = response.json()
        key = "batchNumber"
        value = batchNumber
        scan[key] = value
        scans.append(scan)
        print('Created Scan ' + str(batchNumber) + ' (ID ' + scan['id'] + ')') 
print('Created ' + str(batchNumber) + ' scans - moving on to retrieving scans')
for scan in scans:
    scanId = scan['id']
    scanStatus = scan['status']
    batch = scan['batchNumber']
    w = []
    ds = []
    mds = []
    while scanStatus in ['Running','NotStarted']:
        getStatus = fxRequestWithRetry("GET",f'{powerBIAPIBaseUri}/admin/workspaces/scanStatus/{scanId}',"")
        scanStatus = getStatus.json()['status']
        print('Scan ' + str(batch) + ' (ID ' + scanId + ') is ' + scanStatus)
        time.sleep(10)
    if scanStatus == 'Succeeded':
        response =  fxRequestWithRetry("GET",f'{powerBIAPIBaseUri}/admin/workspaces/scanResult/{scanId}',"")
        scanResults = response.json()
        w.append(scanResults.get('workspaces',None))
        ds.append(scanResults.get('datasourceInstances',None))
        mds.append(scanResults.get('misconfiguredDatasourceInstances',None))
        print('Results from Scan ' + str(batch) + ' (ID ' + scanId + ') Retrieved from GetScanResult')
        with open(f'/lakehouse/default/{nameOfWorkspacesFileFolder}Workspaces_{str(batch)}.json', 'w') as f:
                f.write(json.dumps(w))
                f.close()
        if len(ds) > 0:
                with open(f'/lakehouse/default/{nameOfDatasourcesFileFolder}DatasourceInstances_{str(batch)}.json', 'w') as f:
                        f.write(json.dumps(ds))
                        f.close()
        if len(mds) > 0:
                with open(f'/lakehouse/default/{nameOfDatasourcesFileFolder}misconfiguredDatasourceInstances_{str(batch)}.json', 'w') as f:
                        f.write(json.dumps(mds))
                        f.close()
        print('JSON files from Scan ' + str(batch) + ' (ID ' + scanId + ') retrieved and saved to Lakehouse Files')         
print('All Scans Created and Retrieved!') 

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 74, Finished, Available)

Starting processing of 1 workspace batches
Sending Scan 1 to PostWorkspaceInfo API
Created Scan 1 (ID f7e5d56c-a4cd-4c24-adda-0a9e678aade9)
Created 1 scans - moving on to retrieving scans
Scan 1 (ID f7e5d56c-a4cd-4c24-adda-0a9e678aade9) is Running
Scan 1 (ID f7e5d56c-a4cd-4c24-adda-0a9e678aade9) is Succeeded
Results from Scan 1 (ID f7e5d56c-a4cd-4c24-adda-0a9e678aade9) Retrieved from GetScanResult
JSON files from Scan 1 (ID f7e5d56c-a4cd-4c24-adda-0a9e678aade9) retrieved and saved to Lakehouse Files
All Scans Created and Retrieved!


In [73]:
capacitiesSchema = StructType([
    StructField('id',StringType(),True),  
    StructField('displayName',StringType(),True),  
    StructField('state',StringType(),True),  
    StructField('region',StringType(),True),  
    StructField('sku',StringType(),True)
])
noCapacityJSON = {'id': '00000000-0000-0000-0000-00000000','displayName': 'No Capacity'}
source = fxRequestWithRetry('GET',f'{powerBIAPIBaseUri}/admin/capacities','')
capacities = source.json()['value']
capacities.append(noCapacityJSON)
capacities_raw_df = spark.read.option("multiline","true").json(sc.parallelize(capacities))
capacities_df = capacities_raw_df.select(col('id').alias('CapacityID'), 
        col('displayName').alias('CapacityName'),
        col('state').alias('CapacityState'),
        col('region').alias('CapacityRegion'),
        col('sku').alias('CapacitySKU'))\
    .drop('admins','users')
writeToLake = capacities_df.write.mode("overwrite").format("delta").save("Tables/Capacities")

capacityAdmins_df = capacities_raw_df.select(col('id').alias('CapacityID'), explode("admins").alias("CapacityAdmins"))
writeToLake = capacityAdmins_df.write.mode("overwrite").format("delta").save("Tables/CapacityAdmins")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 75, Finished, Available)

In [74]:
domainsSchema = StructType([
    StructField('id',StringType(),True),  
    StructField('displayName',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('parentDomainId',StringType(),True),  
    StructField('contributorsScope',StringType(),True)
])
noDomainJSON = {'id': '00000000-0000-0000-0000-00000000','displayName': 'No Domain'}
source = fxRequestWithRetry('GET',f'{fabricAPIBaseUri}/admin/domains','')
domains = source.json()['domains']
domains.append(noDomainJSON)
domains_df = spark.createDataFrame(data=domains,schema=domainsSchema)
domains_df = domains_df.select(col('id').alias('DomainID'), 
        col('displayName').alias('DomainName'),
        col('description').alias('DomainDescription'),
        col('parentDomainId').alias('ParentDomainID'),
        col('contributorsScope').alias('ContributorsScope'))
writeToLake = domains_df.write.mode("overwrite").format("delta").save("Tables/Domains")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 76, Finished, Available)

In [75]:
deploymentPipelinesSchema = StructType([
    StructField('id',StringType(),True),  
    StructField('displayName',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('users',ArrayType(StructType([
        StructField('identifier',StringType(),True),  
        StructField('accessRight',StringType(),True),  
        StructField('principalType',StringType(),True)
    ])),True),  
    StructField('stages',ArrayType(StructType([
        StructField('order',StringType(),True),  
        StructField('workspaceId',StringType(),True),  
        StructField('workspaceName',StringType(),True)
    ])),True)
])
noPipelineJSON = {'id': '00000000-0000-0000-0000-00000000','displayName': 'No Pipeline'}
source = fxRequestWithRetry('GET',f'{powerBIAPIBaseUri}/admin/pipelines?$expand=users,stages','')
deploymentPipelines = source.json()['value']
deploymentPipelines.append(noPipelineJSON)
deploymentPipelines_raw_df = spark.createDataFrame(data=deploymentPipelines,schema=deploymentPipelinesSchema)
deploymentPipelines_df = deploymentPipelines_raw_df.select(col('id').alias('DeploymentPipelineID'), 
        col('displayName').alias('DeploymentPipelineName'),
        col('description').alias('DeploymentPipelineDescription'))
writeToLake = deploymentPipelines_df.write.mode("overwrite").format("delta").save("Tables/DeploymentPipelines")

deploymentPipelineStages_df = deploymentPipelines_raw_df.select(col('id').alias('DeploymentPipelineID'),
        'stages')\
    .withColumn('Stages',explode('stages'))\
    .withColumn('DeploymentPipelineStageID',concat(col('DeploymentPipelineID'),lit('_'),col('Stages.order')))\
    .withColumn('DeploymentPipelineStageOrder',col('Stages.order'))\
    .withColumn('DeploymentPipelineStageWorkspaceID',col('Stages.workspaceId'))\
    .drop('Stages','stages')
writeToLake = deploymentPipelineStages_df.write.mode("overwrite").format("delta").save("Tables/DeploymentPipelineStages")

deploymentPipelineStages = spark.sql("SELECT DeploymentPipelineStageWorkspaceID, DeploymentPipelineStageID, DeploymentPipelineID FROM DeploymentPipelineStages")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 77, Finished, Available)

### Define workspace schema and load to raw DataFrame

In [76]:
workspace_schema = StructType([
StructField('id',StringType(),True),  
StructField('name',StringType(),True),  
StructField('description',StringType(),True),  
StructField('type',StringType(),True),  
StructField('state',StringType(),True),  
StructField('capacityMigrationStatus',BooleanType(),True),  
StructField('isOnDedicatedCapacity',BooleanType(),True),  
StructField('capacityId',StringType(),True),  
StructField('defaultDatasetStorageFormat',StringType(),True),  
StructField('domainId',StringType(),True),  
StructField('users',ArrayType(StructType([
    StructField('groupUserAccessRight',StringType(),True),  
    StructField('emailAddress',StringType(),True),  
    StructField('displayName',StringType(),True),  
    StructField('identifier',StringType(),True),  
    StructField('graphId',StringType(),True),  
    StructField('principalType',StringType(),True),  
    StructField('userType',StringType(),True)
    ])),True), 
StructField('reports',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('reportType',StringType(),True),  
    StructField('appId',StringType(),True),  
    StructField('datasetId',StringType(),True),  
    StructField('originalReportObjectId',StringType(),True),
    StructField('createdDateTime',StringType(),True),  
    StructField('modifiedDateTime',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('createdBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('createdById',StringType(),True),
    StructField('users',ArrayType(StructType([
        StructField('reportUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
    ])),True),  
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True)
    ])),True),  
StructField('dashboards',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('appId',StringType(),True),  
    StructField('displayName',StringType(),True),  
    StructField('isReadOnly',BooleanType(),True),
    StructField('users',ArrayType(StructType([
        StructField('dashboardUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
        ])),True),  
    StructField('tiles',ArrayType(StructType([
        StructField('id',StringType(),True),  
        StructField('title',StringType(),True),  
        StructField('reportId',StringType(),True),  
        StructField('datasetId',StringType(),True)
        ])),True),      
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True)
    ])),True),  
StructField('datasets',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),   
    StructField('description',StringType(),True),  
    StructField('configuredBy',StringType(),True),  
    StructField('configuredById',StringType(),True),  
    StructField('isEffectiveIdentityRequired',BooleanType(),True),  
    StructField('isEffectiveIdentityRolesRequired',BooleanType(),True),  
    StructField('targetStorageMode',StringType(),True),  
    StructField('createdDate',StringType(),True),  
    StructField('schemaMayNotBeUpToDate',BooleanType(),True),  
    StructField('schemaRetrievalError',StringType(),True),  
    StructField('contentProviderType',StringType(),True),  
    StructField('refreshSchedule',StructType([
        StructField('days',ArrayType(StringType()),True),  
        StructField('times',ArrayType(StringType()),True),  
        StructField('enabled',BooleanType(),True),  
        StructField('localTimeZoneId',StringType(),True),  
        StructField('notifyOption',StringType(),True)
        ]),True),  
    StructField('directQueryRefreshSchedule',StructType([
        StructField('frequency',IntegerType(),True),  
        StructField('days',ArrayType(StringType()),True),  
        StructField('times',ArrayType(StringType()),True),  
        StructField('localTimeZoneId',StringType(),True)
        ]),True),  
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('upstreamDataflows',ArrayType(StructType([
        StructField('targetDataflowId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True),
    StructField('upstreamDatasets',ArrayType(StructType([
        StructField('targetDatasetId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True),  
    StructField('upstreamDatamarts',ArrayType(StructType([
        StructField('targetDatamartId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True), 
    StructField('datasourceUsages',ArrayType(StructType([
        StructField('datasourceInstanceId',StringType(),True)
        ])),True),  
    StructField('misconfiguredDatasourceUsages',ArrayType(StructType([
        StructField('datasourceInstanceId',StringType(),True)
        ])),True), 
    StructField('users',ArrayType(StructType([
        StructField('datasetUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
        ])),True),
    StructField('roles',ArrayType(StructType([
        StructField('name',StringType(),True),  
        StructField('modelPermission',StringType(),True),  
        StructField('members',ArrayType(StructType([
            StructField('memberName',StringType(),True),  
            StructField('memberId',StringType(),True),  
            StructField('memberType',StringType(),True),  
            StructField('identityProvider',StringType(),True)
        ])),True),  
        StructField('tablePermissions',ArrayType(StructType([
            StructField('name',StringType(),True),  
            StructField('filterExpression',StringType(),True)
        ])),True)
    ])),True),
    StructField('tables',ArrayType(StructType([
        StructField('name',StringType(),True),   
        StructField('description',StringType(),True),
        StructField('isHidden',BooleanType(),True),  
        StructField('storageMode',StringType(),True),  
        StructField('columns',ArrayType(StructType([
            StructField('name',StringType(),True),   
            StructField('description',StringType(),True),
            StructField('dataType',StringType(),True),  
            StructField('isHidden',BooleanType(),True),  
            StructField('columnType',StringType(),True)
            ])),True),  
        StructField('measures',ArrayType(StructType([
            StructField('name',StringType(),True),  
            StructField('expression',StringType(),True),  
            StructField('description',StringType(),True),  
            StructField('isHidden',BooleanType(),True)
            ])),True),  
        StructField('source',ArrayType(StructType([
            StructField('expression',StringType(),True)
            ])),True)
        ])),True),  
    StructField('expressions',ArrayType(StructType([
        StructField('name',StringType(),True),  
        StructField('expression',StringType(),True)
        ])),True),  
    StructField('relations',ArrayType(StructType([
        StructField('dependentOnArtifactId',StringType(),True),  
        StructField('workspaceId',StringType(),True),  
        StructField('relationType',StringType(),True),  
        StructField('settingsList',StringType(),True),  
        StructField('usage',StringType(),True)
        ])),True)
    ])),True), 
StructField('KQLDatabase',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('state',StringType(),True),    
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),  
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('lastUpdatedDate',StringType(),True),  
    StructField('createdDate',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('createdBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('createdById',StringType(),True),  
    StructField('users',ArrayType(StructType([
        StructField('artifactUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
    ])),True),  
    StructField('relations',ArrayType(StructType([
        StructField('dependentOnArtifactId',StringType(),True),  
        StructField('workspaceId',StringType(),True),  
        StructField('relationType',StringType(),True),  
        StructField('settingsList',StringType(),True),  
        StructField('usage',StringType(),True)
        ])),True),  
    StructField('extendedProperties',StructType([
        StructField('QueryServiceUri',StringType(),True),  
        StructField('IngestionServiceUri',StringType(),True),  
        StructField('Region',StringType(),True)
        ]),True)
    ])),True),  
StructField('KQLQueryset',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('state',StringType(),True),  
    StructField('lastUpdatedDate',StringType(),True),  
    StructField('createdDate',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('createdBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('createdById',StringType(),True),  
    StructField('users',ArrayType(StructType([
        StructField('artifactUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
    ])),True),   
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),  
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('relations',ArrayType(StructType([
        StructField('dependentOnArtifactId',StringType(),True),  
        StructField('workspaceId',StringType(),True),  
        StructField('relationType',StringType(),True),  
        StructField('settingsList',StringType(),True),  
        StructField('usage',StringType(),True)
        ])),True),  
    StructField('extendedProperties',StructType([   
        ]),True)
    ])),True),  
StructField('Lakehouse',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('state',StringType(),True),  
    StructField('lastUpdatedDate',StringType(),True),  
    StructField('createdDate',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('createdBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('createdById',StringType(),True), 
    StructField('users',ArrayType(StructType([
        StructField('artifactUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
    ])),True),  
    StructField('extendedProperties',StructType([
        StructField('OneLakeTablesPath',StringType(),True),  
        StructField('OneLakeFilesPath',StringType(),True),  
        StructField('DwProperties',StringType(),True)
        ]),True)
    ])),True),  
StructField('DataPipeline',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('state',StringType(),True),  
    StructField('lastUpdatedDate',StringType(),True),  
    StructField('createdDate',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('createdBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('createdById',StringType(),True),
    StructField('users',ArrayType(StructType([
        StructField('artifactUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
    ])),True),   
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),  
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('extendedProperties',StructType([      
        ]),True)
    ])),True),  
StructField('Notebook',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('state',StringType(),True),  
    StructField('lastUpdatedDate',StringType(),True),  
    StructField('createdDate',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('createdBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('createdById',StringType(),True),
    StructField('users',ArrayType(StructType([
        StructField('artifactUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
    ])),True),   
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),  
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('relations',ArrayType(StructType([
        StructField('dependentOnArtifactId',StringType(),True),  
        StructField('workspaceId',StringType(),True),  
        StructField('relationType',StringType(),True),  
        StructField('settingsList',StringType(),True),  
        StructField('usage',StringType(),True)])),True),  
        StructField('extendedProperties',StructType([
        ]),True)
    ])),True),   
StructField('dataflows',ArrayType(StructType([
    StructField('objectId',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('configuredBy',StringType(),True),  
    StructField('configuredById',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('modifiedDateTime',StringType(),True), 
    StructField('modelUrl',StringType(),True), 
    StructField('description',StringType(),True), 
    StructField('refreshSchedule',StructType([
        StructField('days',ArrayType(StringType()),True),  
        StructField('times',ArrayType(StringType()),True),  
        StructField('enabled',BooleanType(),True),  
        StructField('localTimeZoneId',StringType(),True),  
        StructField('notifyOption',StringType(),True)
        ]),True),  
    StructField('users',ArrayType(StructType([
        StructField('dataflowUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
        ])),True), 
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('upstreamDataflows',ArrayType(StructType([
        StructField('targetDataflowId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True),
    StructField('upstreamDatamarts',ArrayType(StructType([
        StructField('targetDatamartId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True), 
    StructField('datasourceUsages',ArrayType(StructType([
        StructField('datasourceInstanceId',StringType(),True)
        ])),True),  
    StructField('misconfiguredDatasourceUsages',ArrayType(StructType([
        StructField('datasourceInstanceId',StringType(),True)
        ])),True), 
    StructField('generation',IntegerType(),True)
    ])),True),  
StructField('datamarts',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('description',StringType(),True),  
    StructField('type',StringType(),True),  
    StructField('state',StringType(),True),  
    StructField('status',StringType(),True),  
    StructField('suspendedBatchId',StringType(),True),  
    StructField('configuredBy',StringType(),True),  
    StructField('configuredById',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('modifiedDateTime',StringType(),True),  
    StructField('users',ArrayType(StructType([
        StructField('datamartUserAccessRight',StringType(),True),  
        StructField('emailAddress',StringType(),True),  
        StructField('displayName',StringType(),True),  
        StructField('identifier',StringType(),True),  
        StructField('graphId',StringType(),True),  
        StructField('principalType',StringType(),True),  
        StructField('userType',StringType(),True)
        ])),True), 
    StructField('upstreamDataflows',ArrayType(StructType([
        StructField('targetDataflowId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True),
    StructField('upstreamDatamarts',ArrayType(StructType([
        StructField('targetDatamartId',StringType(),True),  
        StructField('groupId',StringType(),True)
        ])),True), 
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('datasourceUsages',ArrayType(StructType([
        StructField('datasourceInstanceId',StringType(),True)
        ])),True),
    StructField('misconfiguredDatasourceUsages',ArrayType(StructType([
        StructField('datasourceInstanceId',StringType(),True)
        ])),True)
    ])),True),  
StructField('warehouses',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('configuredBy',StringType(),True),  
    StructField('configuredById',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('modifiedDateTime',StringType(),True)
    ])),True),  
StructField('SQLAnalyticsEndpoint',ArrayType(StructType([
    StructField('id',StringType(),True),  
    StructField('name',StringType(),True),  
    StructField('sensitivityLabel',StructType([
        StructField('labelId',StringType(),True)
        ]),True),
    StructField('endorsementDetails',StructType([
        StructField('certifiedBy',StringType(),True),
        StructField('endorsement',StringType(),True)
        ]),True),
    StructField('configuredBy',StringType(),True),  
    StructField('configuredById',StringType(),True),  
    StructField('modifiedBy',StringType(),True),  
    StructField('modifiedById',StringType(),True),  
    StructField('modifiedDateTime',StringType(),True),  
    StructField('relations',ArrayType(StructType([
        StructField('dependentOnArtifactId',StringType(),True),  
        StructField('workspaceId',StringType(),True),  
        StructField('relationType',StringType(),True),  
        StructField('settingsList',StringType(),True),  
        StructField('usage',StringType(),True)
        ])),True)
    ])),True)
])

unknownWorkspace_JSON = [
  {
    'id': '00000000-0000-0000-0000-00000000',
    'name': 'No Workspace',
    'capacityId': '00000000-0000-0000-0000-00000000',
    'domainId': '00000000-0000-0000-0000-00000000',
    'reports': [
      {
        'id': '00000000-0000-0000-0000-00000000',
        'name': 'No Report',
        'reportType': 'N/A',
        'appId': '00000000-0000-0000-0000-00000000',
        'datasetId': '00000000-0000-0000-0000-00000000',
        'originalReportObjectId': '00000000-0000-0000-0000-00000000'
      }
    ],
    'dashboards': [
      {
        'id': '00000000-0000-0000-0000-00000000',
        'appId': '00000000-0000-0000-0000-00000000',
        'displayName': 'No Dashboard',
        'tiles': [
          {
            'id': '00000000-0000-0000-0000-00000000',
            'title': 'No Tile',
            'reportId': '00000000-0000-0000-0000-00000000',
            'datasetId': '00000000-0000-0000-0000-00000000'
          }
        ]
      }
    ],
    'datasets': [
      {
        'id': '00000000-0000-0000-0000-00000000',
        'name': 'No Semantic Model'
      }
    ],
    'dataflows': [
      {
        'objectId': '00000000-0000-0000-0000-00000000',
        'name': 'No Dataflow'
      }
    ],
    'datamarts': [
      {
        'id': '00000000-0000-0000-0000-00000000',
        'name': 'No Datamart'
      }
    ]
  }
]

with open(f'/lakehouse/default/{nameOfControlFileFolder}/EmptyWorkspace.json', 'w') as f:
    f.write(json.dumps(unknownWorkspace_JSON))
    f.close()

emptyWorkspace_df_raw = spark.read.json(f'{nameOfControlFileFolder}/EmptyWorkspace.json',schema = workspace_schema)
workspace_df_raw = spark.read.json(f'{nameOfWorkspacesFileFolder}/*.json',schema = workspace_schema)

workspace_df_raw = workspace_df_raw.union(emptyWorkspace_df_raw) 

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 78, Finished, Available)

##### Create Workspaces DataFrame, save to temp parquet file and merge with Delta table

In [77]:
workspace_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
.select(col('id').alias('WorkspaceID'), 
    col('DeploymentPipelineID'),
    col('DeploymentPipelineStageID'),
    col('capacityId').alias('CapacityID'), 
    col('domainId').alias('DomainID'), 
    col('name').alias('WorkspaceName'), 
    col('description').alias('WorkspaceDescription'),
    col('type').alias('WorkspaceType'),
    col('state').alias('WorkspaceState'),
    col('capacityMigrationStatus').alias('CapacityMigrationStatus'),
    col('isOnDedicatedCapacity').alias('IsOnDedicatedCapacity'))\
.withColumn("LastWorkspaceScan",lit(modifiedSince).cast('timestamp'))\
.fillna({'DomainID':'00000000-0000-0000-0000-00000000',
    'CapacityID':'00000000-0000-0000-0000-00000000',
    'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
    'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
.write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/Workspaces') 

fxMergeDeltaTable('Workspaces','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 79, Finished, Available)

'Workspaces is now up to date'

##### Create WorkspacePermissions DataFrame, save to temp parquet file and merge with Delta table

In [78]:
workspacePermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
.select(col('id').alias('WorkspaceID'), 
    col('capacityId').alias('CapacityID'), 
    col('domainId').alias('DomainID'),
    col('DeploymentPipelineID'),
    col('DeploymentPipelineStageID'),
    'users')\
.withColumn('WorkspacePermissions',explode('users'))\
.withColumn('Permissions',col('WorkspacePermissions.groupUserAccessRight'))\
.withColumn('EmailAddress',col('WorkspacePermissions.emailAddress'))\
.withColumn('DisplayName',col('WorkspacePermissions.displayName'))\
.withColumn('Identifier',col('WorkspacePermissions.identifier'))\
.withColumn('GraphID',col('WorkspacePermissions.graphId'))\
.withColumn('PrincipalType',col('WorkspacePermissions.principalType'))\
.withColumn('UserType',col('WorkspacePermissions.userType'))\
.drop('users','WorkspacePermissions')\
.fillna({'DomainID':'00000000-0000-0000-0000-00000000',
    'CapacityID':'00000000-0000-0000-0000-00000000',
    'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
    'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
.write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/WorkspacePermissions') 

#workspacePermissions_df.show()
fxMergeDeltaTable('WorkspacePermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 80, Finished, Available)

'WorkspacePermissions is now up to date'

In [79]:
semanticModels_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelName',col('SemanticModels.name'))\
    .withColumn('SemanticModelDescription',col('SemanticModels.description'))\
    .withColumn('SemanticModelConfiguredBy',col('SemanticModels.configuredBy'))\
    .withColumn('SemanticModelConfiguredByGraphID',col('SemanticModels.configuredById'))\
    .withColumn('IsEffectiveIdentityRequired',col('SemanticModels.isEffectiveIdentityRequired'))\
    .withColumn('IsEffectiveIdentityRolesRequired',col('SemanticModels.isEffectiveIdentityRolesRequired'))\
    .withColumn('TargetStorageMode',col('SemanticModels.targetStorageMode'))\
    .withColumn('ContentProviderType',col('SemanticModels.contentProviderType'))\
    .withColumn('SchemaMayNotBeUpToDate',col('SemanticModels.schemaMayNotBeUpToDate'))\
    .withColumn('SchemaRetrievalError',col('SemanticModels.schemaRetrievalError'))\
    .withColumn('SemanticModelCreatedDatetime',col('SemanticModels.createdDate').cast('timestamp'))\
    .withColumn('SensitivityLabelID',col('SemanticModels.sensitivityLabel.labelId'))\
    .withColumn('Endorsement',col('SemanticModels.endorsementDetails.endorsement'))\
    .withColumn('EndorsementCertifiedBy',col('SemanticModels.endorsementDetails.certifiedBy'))\
    .withColumn('RefreshScheduleDays',concat_ws(',',coalesce(col('SemanticModels.refreshSchedule.days'),col('SemanticModels.directQueryRefreshSchedule.days'))))\
    .withColumn('RefreshScheduleTimes',concat_ws(',',coalesce(col('SemanticModels.refreshSchedule.times'),col('SemanticModels.directQueryRefreshSchedule.times'))))\
    .withColumn('RefreshScheduleEnabled',col('SemanticModels.refreshSchedule.enabled'))\
    .withColumn('RefreshScheduleTimeZoneID',concat_ws(',',coalesce(col('SemanticModels.refreshSchedule.localTimeZoneId'),col('SemanticModels.directQueryRefreshSchedule.localTimeZoneId'))))\
    .withColumn('RefreshScheduleNotifyOption',col('SemanticModels.refreshSchedule.notifyOption'))\
    .withColumn('DirectQueryRefreshScheduleFrequency',col('SemanticModels.directQueryRefreshSchedule.frequency'))\
    .replace('Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday','Everyday','RefreshScheduleDays')\
    .replace('Monday,Tuesday,Wednesday,Thursday,Friday','Every Weekday','RefreshScheduleDays')\
    .drop('sensitivityLabel','refreshSchedule','directQueryRefreshSchedule','endorsementDetails','datasets','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModels') 
    
fxMergeDeltaTable('SemanticModels','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 81, Finished, Available)

'SemanticModels is now up to date'

In [80]:
semanticModelPermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelPermissions',explode('SemanticModels.users'))\
    .withColumn('Permissions',col('SemanticModelPermissions.datasetUserAccessRight'))\
    .withColumn('EmailAddress',col('SemanticModelPermissions.emailAddress'))\
    .withColumn('DisplayName',col('SemanticModelPermissions.displayName'))\
    .withColumn('Identifier',col('SemanticModelPermissions.identifier'))\
    .withColumn('GraphID',col('SemanticModelPermissions.graphId'))\
    .withColumn('PrincipalType',col('SemanticModelPermissions.principalType'))\
    .withColumn('UserType',col('SemanticModelPermissions.userType'))\
    .drop('datasets','SemanticModelPermissions','SemanticModels')\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelPermissions') 
    
fxMergeDeltaTable('SemanticModelPermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 82, Finished, Available)

'SemanticModelPermissions is now up to date'

In [81]:
semanticModelMisconfiguredDatasources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelDatasources',explode('SemanticModels.misconfiguredDatasourceUsages'))\
    .withColumn('DatasourceID',col('SemanticModelDatasources.datasourceInstanceId'))\
    .drop('datasets','SemanticModelDatasources','SemanticModels')

semanticModelConfiguredDatasources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelDatasources',explode('SemanticModels.datasourceUsages'))\
    .withColumn('DatasourceID',col('SemanticModelDatasources.datasourceInstanceId'))\
    .drop('datasets','SemanticModelDatasources','SemanticModels')

semanticModelDatasources_df = semanticModelMisconfiguredDatasources_df.union(semanticModelConfiguredDatasources_df)\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelDatasources') 

fxMergeDeltaTable('SemanticModelDatasources','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 83, Finished, Available)

'SemanticModelDatasources is now up to date'

In [82]:
semanticModelDataflowLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelUpstreamDataflows',explode('SemanticModels.upstreamDataflows'))\
    .withColumn('TargetDataflowID',col('SemanticModelUpstreamDataflows.targetDataflowId'))\
    .withColumn('TargetDataflowWorkspaceID',col('SemanticModelUpstreamDataflows.groupId'))\
    .drop('datasets','SemanticModelUpstreamDataflows','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelDataflowLinks') 
    
fxMergeDeltaTable('SemanticModelDataflowLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 84, Finished, Available)

'SemanticModelDataflowLinks is now up to date'

In [83]:
semanticModelDatamartLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelUpstreamDatamarts',explode('SemanticModels.upstreamDatamarts'))\
    .withColumn('TargetDatamartID',col('SemanticModelUpstreamDatamarts.targetDatamartId'))\
    .withColumn('TargetDatamartWorkspaceID',col('SemanticModelUpstreamDatamarts.groupId'))\
    .drop('datasets','SemanticModelUpstreamDatamarts','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelDatamartLinks') 
    
fxMergeDeltaTable('SemanticModelDatamartLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 85, Finished, Available)

'SemanticModelDatamartLinks is now up to date'

In [84]:
semanticModelSemanticModelLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelUpstreamDatasets',explode('SemanticModels.upstreamDatasets'))\
    .withColumn('TargetSemanticModelID',col('SemanticModelUpstreamDatasets.targetDatasetId'))\
    .withColumn('TargetSemanticModelWorkspaceID',col('SemanticModelUpstreamDatasets.groupId'))\
    .drop('datasets','SemanticModelUpstreamDatasets','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelSemanticModelLinks') 
    
fxMergeDeltaTable('SemanticModelSemanticModelLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 86, Finished, Available)

'SemanticModelSemanticModelLinks is now up to date'

In [85]:
semanticModelTables_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelTables',explode('SemanticModels.tables'))\
    .withColumn('TableNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelTables.name')))\
    .withColumn('TableName',col('SemanticModelTables.name'))\
    .withColumn('TableDescription',col('SemanticModelTables.description'))\
    .withColumn('IsHidden',col('SemanticModelTables.isHidden'))\
    .withColumn('StorageMode',col('SemanticModelTables.storageMode'))\
    .drop('datasets','SemanticModelTables','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelTables') 
    
fxMergeDeltaTable('SemanticModelTables','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 87, Finished, Available)

'SemanticModelTables is now up to date'

In [86]:
semanticModelColumns_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelTables',explode('SemanticModels.tables'))\
    .withColumn('TableNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelTables.name')))\
    .withColumn('SemanticModelColumns',explode('SemanticModelTables.columns'))\
    .withColumn('ColumnName',col('SemanticModelColumns.name'))\
    .withColumn('IsHidden',col('SemanticModelColumns.isHidden'))\
    .withColumn('ColumnDescription',col('SemanticModelColumns.description'))\
    .withColumn('ColumnType',col('SemanticModelColumns.columnType'))\
    .withColumn('ColumnDataType',col('SemanticModelColumns.dataType'))\
    .drop('datasets','SemanticModelTables','SemanticModelColumns','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelColumns') 
    
fxMergeDeltaTable('SemanticModelColumns','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 88, Finished, Available)

'SemanticModelColumns is now up to date'

In [87]:
semanticModelMeasures_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelTables',explode('SemanticModels.tables'))\
    .withColumn('TableNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelTables.name')))\
    .withColumn('SemanticModelMeasures',explode('SemanticModelTables.measures'))\
    .withColumn('MeasureName',col('SemanticModelMeasures.name'))\
    .withColumn('IsHidden',col('SemanticModelMeasures.isHidden'))\
    .withColumn('MeasureDescription',col('SemanticModelMeasures.description'))\
    .withColumn('MeasureExpression',col('SemanticModelMeasures.expression'))\
    .drop('datasets','SemanticModelTables','SemanticModelMeasures','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelMeasures') 
    
fxMergeDeltaTable('SemanticModelMeasures','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 89, Finished, Available)

'SemanticModelMeasures is now up to date'

In [88]:
semanticModelTableSources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelTables',explode('SemanticModels.tables'))\
    .withColumn('TableNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelTables.name')))\
    .withColumn('SemanticModelTableSources',explode('SemanticModelTables.source'))\
    .withColumn('Expression',col('SemanticModelTableSources.expression'))\
    .drop('datasets','SemanticModelTables','SemanticModelTableSources','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelTableSources') 
    
fxMergeDeltaTable('SemanticModelTableSources','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 90, Finished, Available)

'SemanticModelTableSources is now up to date'

In [89]:
semanticModelRoles_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelRoles',explode('SemanticModels.roles'))\
    .withColumn('RoleNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelRoles.name')))\
    .withColumn('RoleName',col('SemanticModelRoles.name'))\
    .withColumn('ModelPermission',col('SemanticModelRoles.modelPermission'))\
    .drop('datasets','SemanticModelRoles''SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelRoles') 
    
fxMergeDeltaTable('SemanticModelRoles','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 91, Finished, Available)

'SemanticModelRoles is now up to date'

In [90]:
df = spark.sql("SELECT * FROM FabricReporting.SemanticModelRoleMembers LIMIT 1000")
display(df)

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 92, Finished, Available)

SynapseWidget(Synapse.DataFrame, e1953f3f-9e77-4817-a899-1fa3f9b1aa38)

In [91]:
semanticModelRoleMembers_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelRoles',explode('SemanticModels.roles'))\
    .withColumn('RoleNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelRoles.name')))\
    .withColumn('SemanticModelRoleMembers',explode('SemanticModelRoles.members'))\
    .withColumn('RoleMemberName',col('SemanticModelRoleMembers.memberName'))\
    .withColumn('RoleMemberGraphID',col('SemanticModelRoleMembers.memberId'))\
    .withColumn('RoleMemberType',col('SemanticModelRoleMembers.memberType'))\
    .withColumn('RoleMemberIdentityProvider',col('SemanticModelRoleMembers.identityProvider'))\
    .drop('datasets','SemanticModelRoles','SemanticModelRoleMembers','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelRoleMembers') 
    
fxMergeDeltaTable('SemanticModelRoleMembers','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 93, Finished, Available)

'SemanticModelRoleMembers is now up to date'

In [92]:
semanticModelRoleTablePermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datasets')\
    .withColumn('SemanticModels',explode('datasets'))\
    .withColumn('SemanticModelID',col('SemanticModels.id'))\
    .withColumn('SemanticModelRoles',explode('SemanticModels.roles'))\
    .withColumn('RoleNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelRoles.name')))\
    .withColumn('SemanticModelRoleTablePermissions',explode('SemanticModelRoles.tablePermissions'))\
    .withColumn('RoleTableNameDistinct',concat(col('SemanticModels.id'),col('SemanticModelRoles.name')))\
    .withColumn('RoleTableName',col('SemanticModelRoleTablePermissions.name'))\
    .withColumn('RoleTableFilterExpression',col('SemanticModelRoleTablePermissions.filterExpression'))\
    .drop('datasets','SemanticModelRoles','SemanticModelRoleTablePermissions','SemanticModels')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/SemanticModelRoleTablePermissions') 
    
fxMergeDeltaTable('SemanticModelRoleTablePermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 94, Finished, Available)

'SemanticModelRoleTablePermissions is now up to date'

In [93]:
dataflows_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowName',col('Dataflows.name'))\
    .withColumn('DataflowConfiguredBy',col('Dataflows.configuredBy'))\
    .withColumn('DataflowConfiguredByGraphID',col('Dataflows.configuredById'))\
    .withColumn('DataflowModifiedBy',col('Dataflows.modifiedBy'))\
    .withColumn('DataflowModifiedByGraphID',col('Dataflows.modifiedById'))\
    .withColumn('DataflowModifiedDatetime',col('Dataflows.modifiedDateTime').cast('timestamp'))\
    .withColumn('DataflowDescription',col('Dataflows.description'))\
    .withColumn('DataflowModelURL',col('Dataflows.modelUrl'))\
    .withColumn('SensitivityLabelID',col('Dataflows.sensitivityLabel.labelId'))\
    .withColumn('Endorsement',col('Dataflows.endorsementDetails.endorsement'))\
    .withColumn('EndorsementCertifiedBy',col('Dataflows.endorsementDetails.certifiedBy'))\
    .withColumn('RefreshScheduleDays',concat_ws(',',col('Dataflows.refreshSchedule.days')))\
    .withColumn('RefreshScheduleTimes',concat_ws(',',col('Dataflows.refreshSchedule.times')))\
    .withColumn('RefreshScheduleEnabled',col('Dataflows.refreshSchedule.enabled'))\
    .withColumn('RefreshScheduleTimeZoneID',concat_ws(',',col('Dataflows.refreshSchedule.localTimeZoneId')))\
    .withColumn('RefreshScheduleNotifyOption',col('Dataflows.refreshSchedule.notifyOption'))\
    .replace('Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday','Everyday','RefreshScheduleDays')\
    .replace('Monday,Tuesday,Wednesday,Thursday,Friday','Every Weekday','RefreshScheduleDays')\
    .drop('sensitivityLabel','refreshSchedule','endorsementDetails','dataflows','Dataflows')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/Dataflows') 
    
fxMergeDeltaTable('Dataflows','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 95, Finished, Available)

'Dataflows is now up to date'

In [94]:
dataflowPermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowPermissions',explode('Dataflows.users'))\
    .withColumn('Permissions',col('DataflowPermissions.dataflowUserAccessRight'))\
    .withColumn('EmailAddress',col('DataflowPermissions.emailAddress'))\
    .withColumn('DisplayName',col('DataflowPermissions.displayName'))\
    .withColumn('Identifier',col('DataflowPermissions.identifier'))\
    .withColumn('GraphID',col('DataflowPermissions.graphId'))\
    .withColumn('PrincipalType',col('DataflowPermissions.principalType'))\
    .withColumn('UserType',col('DataflowPermissions.userType'))\
    .drop('dataflows','DataflowPermissions','Dataflows')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DataflowPermissions') 
    
fxMergeDeltaTable('DataflowPermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 96, Finished, Available)

'DataflowPermissions is now up to date'

In [95]:
dataflowMisconfiguredDatasources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowDatasources',explode('Dataflows.misconfiguredDatasourceUsages'))\
    .withColumn('DatasourceID',col('DataflowDatasources.datasourceInstanceId'))\
    .drop('dataflows','DataflowDatasources','Dataflows')

dataflowConfiguredDatasources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowDatasources',explode('Dataflows.datasourceUsages'))\
    .withColumn('DatasourceID',col('DataflowDatasources.datasourceInstanceId'))\
    .drop('dataflows','DataflowDatasources','Dataflows')

dataflowDatasources_df = dataflowMisconfiguredDatasources_df.union(dataflowConfiguredDatasources_df)\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DataflowDatasources') 

fxMergeDeltaTable('DataflowDatasources','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 97, Finished, Available)

'DataflowDatasources is now up to date'

In [96]:
dataflowDataflowLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowUpstreamDataflows',explode('Dataflows.upstreamDataflows'))\
    .withColumn('TargetDataflowID',col('DataflowUpstreamDataflows.targetDataflowId'))\
    .withColumn('TargetDataflowWorkspaceID',col('DataflowUpstreamDataflows.groupId'))\
    .drop('dataflows','DataflowUpstreamDataflows','Dataflows')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DataflowDataflowLinks') 
    
fxMergeDeltaTable('DataflowDataflowLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 98, Finished, Available)

'DataflowDataflowLinks is now up to date'

In [97]:
dataflowDataflowLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowUpstreamDataflows',explode('Dataflows.upstreamDataflows'))\
    .withColumn('TargetDataflowID',col('DataflowUpstreamDataflows.targetDataflowId'))\
    .withColumn('TargetDataflowWorkspaceID',col('DataflowUpstreamDataflows.groupId'))\
    .drop('dataflows','DataflowUpstreamDataflows','Dataflows')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DataflowDataflowLinks') 
    
fxMergeDeltaTable('DataflowDataflowLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 99, Finished, Available)

'DataflowDataflowLinks is now up to date'

In [98]:
dataflowDatamartLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dataflows')\
    .withColumn('Dataflows',explode('dataflows'))\
    .withColumn('DataflowID',col('Dataflows.objectId'))\
    .withColumn('DataflowUpstreamDatamarts',explode('Dataflows.upstreamDatamarts'))\
    .withColumn('TargetDatamartID',col('DataflowUpstreamDatamarts.targetDatamartId'))\
    .withColumn('TargetDatamartWorkspaceID',col('DataflowUpstreamDatamarts.groupId'))\
    .drop('dataflows','DataflowUpstreamDatamarts','Dataflows')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DataflowDatamartLinks') 
    
fxMergeDeltaTable('DataflowDatamartLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 100, Finished, Available)

'DataflowDatamartLinks is now up to date'

In [99]:
datamarts_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datamarts')\
    .withColumn('Datamarts',explode('datamarts'))\
    .withColumn('DatamartID',col('Datamarts.id'))\
    .withColumn('DatamartName',col('Datamarts.name'))\
    .withColumn('DatamartDescription',col('Datamarts.description'))\
    .withColumn('DatamartType',col('Datamarts.type'))\
    .withColumn('DatamartState',col('Datamarts.state'))\
    .withColumn('DatamartStatus',col('Datamarts.status'))\
    .withColumn('DatamartSuspendedBatchID',col('Datamarts.suspendedBatchId'))\
    .withColumn('DatamartConfiguredBy',col('Datamarts.configuredBy'))\
    .withColumn('DatamartConfiguredByGraphID',col('Datamarts.configuredById'))\
    .withColumn('DatamartModifiedBy',col('Datamarts.modifiedBy'))\
    .withColumn('DatamartModifiedDatetime',col('Datamarts.modifiedById').cast('timestamp'))\
    .withColumn('SensitivityLabelID',col('Datamarts.sensitivityLabel.labelId'))\
    .withColumn('Endorsement',col('Datamarts.endorsementDetails.endorsement'))\
    .withColumn('EndorsementCertifiedBy',col('Datamarts.endorsementDetails.certifiedBy'))\
    .drop('sensitivityLabel','endorsementDetails','datamarts','Datamarts')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/Datamarts') 
    
fxMergeDeltaTable('Datamarts','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 101, Finished, Available)

'Datamarts is now up to date'

In [100]:
datamartPermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datamarts')\
    .withColumn('Datamarts',explode('datamarts'))\
    .withColumn('DatamartID',col('Datamarts.id'))\
    .withColumn('DatamartPermissions',explode('Datamarts.users'))\
    .withColumn('Permissions',col('DatamartPermissions.datamartUserAccessRight'))\
    .withColumn('EmailAddress',col('DatamartPermissions.emailAddress'))\
    .withColumn('DisplayName',col('DatamartPermissions.displayName'))\
    .withColumn('Identifier',col('DatamartPermissions.identifier'))\
    .withColumn('GraphID',col('DatamartPermissions.graphId'))\
    .withColumn('PrincipalType',col('DatamartPermissions.principalType'))\
    .withColumn('UserType',col('DatamartPermissions.userType'))\
    .drop('datamarts','DatamartPermissions','Datamarts')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DatamartPermissions') 
    
fxMergeDeltaTable('DatamartPermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 102, Finished, Available)

'DatamartPermissions is now up to date'

In [101]:
datamartDataflowLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datamarts')\
    .withColumn('Datamarts',explode('datamarts'))\
    .withColumn('DatamartID',col('Datamarts.id'))\
    .withColumn('DatamartUpstreamDataflows',explode('Datamarts.upstreamDataflows'))\
    .withColumn('TargetDataflowID',col('DatamartUpstreamDataflows.targetDataflowId'))\
    .withColumn('TargetDataflowWorkspaceID',col('DatamartUpstreamDataflows.groupId'))\
    .drop('datamarts','DatamartUpstreamDataflows','Datamarts')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DatamartDataflowLinks') 
    
fxMergeDeltaTable('DatamartDataflowLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 103, Finished, Available)

'DatamartDataflowLinks is now up to date'

In [102]:
datamartDatamartLinks_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datamarts')\
    .withColumn('Datamarts',explode('datamarts'))\
    .withColumn('DatamartID',col('Datamarts.id'))\
    .withColumn('DatamartUpstreamDatamarts',explode('Datamarts.upstreamDatamarts'))\
    .withColumn('TargetDatamartID',col('DatamartUpstreamDatamarts.targetDatamartId'))\
    .withColumn('TargetDatamartWorkspaceID',col('DatamartUpstreamDatamarts.groupId'))\
    .drop('datamarts','DatamartUpstreamDatamarts','Datamarts')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DatamartDatamartLinks') 
    
fxMergeDeltaTable('DatamartDatamartLinks','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 104, Finished, Available)

'DatamartDatamartLinks is now up to date'

In [103]:
datamartMisconfiguredDatasources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datamarts')\
    .withColumn('Datamarts',explode('datamarts'))\
    .withColumn('DatamartID',col('Datamarts.id'))\
    .withColumn('DatamartDatasources',explode('Datamarts.misconfiguredDatasourceUsages'))\
    .withColumn('DatasourceID',col('DatamartDatasources.datasourceInstanceId'))\
    .drop('datamarts','DatamartDatasources','Datamarts')

datamartConfiguredDatasources_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'datamarts')\
    .withColumn('Datamarts',explode('datamarts'))\
    .withColumn('DatamartID',col('Datamarts.id'))\
    .withColumn('DatamartDatasources',explode('Datamarts.datasourceUsages'))\
    .withColumn('DatasourceID',col('DatamartDatasources.datasourceInstanceId'))\
    .drop('datamarts','DatamartDatasources','Datamarts')

datamartDatasources_df = datamartMisconfiguredDatasources_df.union(datamartConfiguredDatasources_df)\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DatamartDatasources') 

fxMergeDeltaTable('DatamartDatasources','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 105, Finished, Available)

'DatamartDatasources is now up to date'

In [104]:
reports_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'reports')\
    .withColumn('Reports',explode('reports'))\
    .withColumn('SemanticModelID',col('Reports.datasetId'))\
    .withColumn('AppID',col('Reports.appId'))\
    .withColumn('ReportID',col('Reports.id'))\
    .withColumn('OriginalReportID',coalesce(col('Reports.originalReportObjectId'),col('Reports.id')))\
    .withColumn('ReportName',col('Reports.name'))\
    .withColumn('ReportType',col('Reports.reportType'))\
    .withColumn('ReportCreatedBy',col('Reports.createdBy'))\
    .withColumn('ReportCreatedByGraphID',col('Reports.createdById'))\
    .withColumn('ReportCreatedDatetime',col('Reports.createdDateTime').cast('timestamp'))\
    .withColumn('ReportModifiedBy',col('Reports.modifiedBy'))\
    .withColumn('ReportModifiedByGraphID',col('Reports.modifiedById'))\
    .withColumn('ReportModifiedDatetime',col('Reports.modifiedDateTime').cast('timestamp'))\
    .withColumn('SensitivityLabelID',col('Reports.sensitivityLabel.labelId'))\
    .withColumn('Endorsement',col('Reports.endorsementDetails.endorsement'))\
    .withColumn('EndorsementCertifiedBy',col('Reports.endorsementDetails.certifiedBy'))\
    .drop('sensitivityLabel','endorsementDetails','reports','Reports')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/Reports') 
    
fxMergeDeltaTable('Reports','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 106, Finished, Available)

'Reports is now up to date'

In [105]:
reportPermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'reports')\
    .withColumn('Reports',explode('reports'))\
    .withColumn('SemanticModelID',col('Reports.datasetId'))\
    .withColumn('AppID',col('Reports.appId'))\
    .withColumn('ReportID',col('Reports.id'))\
    .withColumn('ReportPermissions',explode('Reports.users'))\
    .withColumn('Permissions',col('ReportPermissions.reportUserAccessRight'))\
    .withColumn('EmailAddress',col('ReportPermissions.emailAddress'))\
    .withColumn('DisplayName',col('ReportPermissions.displayName'))\
    .withColumn('Identifier',col('ReportPermissions.identifier'))\
    .withColumn('GraphID',col('ReportPermissions.graphId'))\
    .withColumn('PrincipalType',col('ReportPermissions.principalType'))\
    .withColumn('UserType',col('ReportPermissions.userType'))\
    .drop('reports','ReportPermissions','Reports')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'AppID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/ReportPermissions') 
    
fxMergeDeltaTable('ReportPermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 107, Finished, Available)

'ReportPermissions is now up to date'

In [106]:
dashboards_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dashboards')\
    .withColumn('Dashboards',explode('dashboards'))\
    .withColumn('DashboardID',col('Dashboards.id'))\
    .withColumn('AppID',col('Dashboards.appId'))\
    .withColumn('DashboardName',col('Dashboards.displayName'))\
    .withColumn('IsReadyOnly',col('Dashboards.isReadOnly'))\
    .withColumn('SensitivityLabelID',col('Dashboards.sensitivityLabel.labelId'))\
    .withColumn('Endorsement',col('Dashboards.endorsementDetails.endorsement'))\
    .withColumn('EndorsementCertifiedBy',col('Dashboards.endorsementDetails.certifiedBy'))\
    .drop('sensitivityLabel','endorsementDetails','dashboards','Dashboards')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000','CapacityID':'00000000-0000-0000-0000-00000000','WorkspaceID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/Dashboards') 
    
fxMergeDeltaTable('Dashboards','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 108, Finished, Available)

'Dashboards is now up to date'

In [107]:
dashboardPermissions_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dashboards')\
    .withColumn('Dashboards',explode('dashboards'))\
    .withColumn('AppID',col('Dashboards.appId'))\
    .withColumn('DashboardID',col('Dashboards.id'))\
    .withColumn('DashboardPermissions',explode('Dashboards.users'))\
    .withColumn('Permissions',col('DashboardPermissions.dashboardUserAccessRight'))\
    .withColumn('EmailAddress',col('DashboardPermissions.emailAddress'))\
    .withColumn('DisplayName',col('DashboardPermissions.displayName'))\
    .withColumn('Identifier',col('DashboardPermissions.identifier'))\
    .withColumn('GraphID',col('DashboardPermissions.graphId'))\
    .withColumn('PrincipalType',col('DashboardPermissions.principalType'))\
    .withColumn('UserType',col('DashboardPermissions.userType'))\
    .drop('dashboards','DashboardPermissions','Dashboards')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'AppID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DashboardPermissions') 
    
fxMergeDeltaTable('DashboardPermissions','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 109, Finished, Available)

'DashboardPermissions is now up to date'

In [108]:
dashboardTiles_df = workspace_df_raw.join(deploymentPipelineStages, workspace_df_raw.id == deploymentPipelineStages.DeploymentPipelineStageWorkspaceID, "left_outer")\
    .select(col('id').alias('WorkspaceID'), 
        col('DeploymentPipelineID'),
        col('DeploymentPipelineStageID'),
        col('capacityId').alias('CapacityID'), 
        col('domainId').alias('DomainID'),
        'dashboards')\
    .withColumn('Dashboards',explode('dashboards'))\
    .withColumn('AppID',col('Dashboards.appId'))\
    .withColumn('DashboardID',col('Dashboards.id'))\
    .withColumn('DashboardTiles',explode('Dashboards.tiles'))\
    .withColumn('SemanticModelID',col('DashboardTiles.datasetId'))\
    .withColumn('ReportID',col('DashboardTiles.reportId'))\
    .withColumn('DashboardTileID',col('DashboardTiles.id'))\
    .withColumn('DashboardTileTitle',col('DashboardTiles.title'))\
    .drop('dashboards','DashboardTiles','Dashboards')\
    .fillna({'DomainID':'00000000-0000-0000-0000-00000000',
        'AppID':'00000000-0000-0000-0000-00000000',
        'CapacityID':'00000000-0000-0000-0000-00000000',
        'WorkspaceID':'00000000-0000-0000-0000-00000000',
        'SemanticModelID':'00000000-0000-0000-0000-00000000',
        'ReportID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineStageID':'00000000-0000-0000-0000-00000000',
        'DeploymentPipelineID':'00000000-0000-0000-0000-00000000'})\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/DashboardTiles') 
    
fxMergeDeltaTable('DashboardTiles','Merge Delete','t.WorkspaceID = s.WorkspaceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 110, Finished, Available)

'DashboardTiles is now up to date'

In [109]:
datasources_schema = StructType([
    StructField('datasourceType',StringType(),True),  
    StructField('connectionDetails',StructType([
        StructField('extensionDataSourceKind',StringType(),True),  
        StructField('extensionDataSourcePath',StringType(),True),
        StructField('url',StringType(),True), 
        StructField('sharePointSiteUrl',StringType(),True),  
        StructField('kind',StringType(),True), 
        StructField('path',StringType(),True), 
        StructField('server',StringType(),True), 
        StructField('database',StringType(),True), 
        StructField('account',StringType(),True), 
        StructField('connectionString',StringType(),True),
        StructField('domain',StringType(),True), 
        StructField('emailAddress',StringType(),True), 
        StructField('loginServer',StringType(),True), 
        StructField('classInfo',StringType(),True)
    ]),True),  
    StructField('datasourceId',StringType(),True),  
    StructField('gatewayId',StringType(),True)])

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 111, Finished, Available)

In [110]:
unkownDatasource_JSON = {'datasourceType':'Unknown',
'connectionDetails': {'path': 'Unknown Datasource'},
'datasourceId': '00000000-0000-0000-0000-00000000',
'gatewayId': '00000000-0000-0000-0000-00000000'}

with open(f'/lakehouse/default/{nameOfDatasourcesFileFolder}/UnknownDatasource.json', 'w') as f:
    f.write(json.dumps(unkownDatasource_JSON))
    f.close()

datasources_df = spark.read.json(f'{nameOfDatasourcesFileFolder}/*.json',schema = datasources_schema)
datasources_df = datasources_df.select(col('datasourceId').alias('DatasourceID'), col('gatewayId').alias('GatewayClusterID'),col('datasourceType'),col('connectionDetails'))\
    .where(col("datasourceId").isNotNull())\
    .withColumn("DatasourceTypeAlt",when(datasources_df.datasourceType == 'Extension',coalesce(col('connectionDetails.extensionDataSourceKind'),col('connectionDetails.kind')))\
        .otherwise(datasources_df.datasourceType))\
    .withColumn("ConnectionInformation",when(datasources_df.datasourceType == "Web", col('connectionDetails.url'))\
        .when(datasources_df.datasourceType == 'Unknown', col('connectionDetails.path'))\
        .when(datasources_df.datasourceType == 'Extension', coalesce(col('connectionDetails.extensionDataSourcePath'),col('connectionDetails.path')))\
        .when(datasources_df.datasourceType.isin(['AnalysisServices','PostgreSql','Sql','MySql','DB2']), concat(col('connectionDetails.server'), lit(' | '), when(col('connectionDetails.database').isNull(), 'N/A').otherwise(col('connectionDetails.database'))))\
        .when(datasources_df.datasourceType.isin(['AzureDataLakeStorage']), concat(col('connectionDetails.server'), lit(" | "), col('connectionDetails.path')))\
        .when(datasources_df.datasourceType.isin(['Teradata','Oracle','SAPHana','Essbase']), col('connectionDetails.server'))\
        .when(datasources_df.datasourceType.isin(['ODBC','PubNub','OleDb']), col('connectionDetails.connectionString'))\
        .when(datasources_df.datasourceType.isin(['File','Folder']), col('connectionDetails.path'))\
        .when(datasources_df.datasourceType.isin(['Salesforce']), concat(col('connectionDetails.loginServer'), lit(" | "), col('connectionDetails.classInfo')))\
        .when(datasources_df.datasourceType.isin(['ActiveDirectory']), col('connectionDetails.domain'))\
        .when(datasources_df.datasourceType.isin(['Exchange']), col('connectionDetails.emailAddress'))\
        .when(datasources_df.datasourceType.isin(['Web','OData','Facebook']), col('connectionDetails.url'))\
        .when(datasources_df.datasourceType.isin(['SharePointList']), coalesce(col('connectionDetails.sharePointSiteUrl'), col('connectionDetails.url')))\
        .when(datasources_df.datasourceType.isin(['AzureTables']), concat(col('connectionDetails.account'), lit(' | '), col('connectionDetails.domain')))\
        .when(datasources_df.datasourceType.isin(['GoogleAnalytics','AdobeAnalytics']), lit('N/A'))\
        .otherwise('Requires Mapping'))\
    .fillna({'GatewayClusterID':'00000000-0000-0000-0000-00000000'})\
    .drop('connectionDetails','datasourceType')\
    .withColumnRenamed('DatasourceTypeAlt','DatasourceType')\
    .distinct()\
    .write.mode("overwrite").parquet(nameOfTempParquetFileFolder + '/Datasources') 

fxMergeDeltaTable('Datasources','Merge Delete','t.DatasourceID = s.DatasourceID')

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 112, Finished, Available)

'Datasources is now up to date'

In [111]:
SemanticModelLineage_df = spark.sql("""
SELECT 
COALESCE(DT.CapacityID,'00000000-0000-0000-0000-00000000') AS DashboardTileCapacityID,
COALESCE(DT.DomainID,'00000000-0000-0000-0000-00000000') AS DashboardTileDomainID,
COALESCE(DT.WorkspaceID,'00000000-0000-0000-0000-00000000') AS DashboardTileWorkspaceID,
COALESCE(DT.DeploymentPipelineID,'00000000-0000-0000-0000-00000000') AS DashboardTileDeploymentPipelineID,
COALESCE(DT.DeploymentPipelineStageID,'00000000-0000-0000-0000-00000000') AS DashboardTileDeploymentPipelineStageID,
COALESCE(DT.SemanticModelID,'00000000-0000-0000-0000-00000000') AS DashboardTileSemanticModelID,
COALESCE(DT.ReportID,'00000000-0000-0000-0000-00000000') AS DashboardTileReportID,
COALESCE(DT.DashboardTileID,'00000000-0000-0000-0000-00000000') AS DashboardTileID,
COALESCE(D.CapacityID,'00000000-0000-0000-0000-00000000') AS DashboardCapacityID,
COALESCE(D.DomainID,'00000000-0000-0000-0000-00000000') AS DashboardDomainID,
COALESCE(D.WorkspaceID,'00000000-0000-0000-0000-00000000') AS DashboardWorkspaceID,
COALESCE(D.DeploymentPipelineID,'00000000-0000-0000-0000-00000000') AS DashboardDeploymentPipelineID,
COALESCE(D.DeploymentPipelineStageID,'00000000-0000-0000-0000-00000000') AS DashboardDeploymentPipelineStageID,
COALESCE(D.AppID,'00000000-0000-0000-0000-00000000') AS DashboardAppID,
COALESCE(D.DashboardID,'00000000-0000-0000-0000-00000000') AS DashboardID,
COALESCE(R.CapacityID,'00000000-0000-0000-0000-00000000') AS ReportCapacityID,
COALESCE(R.DomainID,'00000000-0000-0000-0000-00000000') AS ReportDomainID,
COALESCE(R.WorkspaceID,'00000000-0000-0000-0000-00000000') AS ReportWorkspaceID,
COALESCE(R.DeploymentPipelineID,'00000000-0000-0000-0000-00000000') AS ReportDeploymentPipelineID,
COALESCE(R.DeploymentPipelineStageID,'00000000-0000-0000-0000-00000000') AS ReportDeploymentPipelineStageID,
COALESCE(R.AppID,'00000000-0000-0000-0000-00000000') AS ReportAppID,
COALESCE(R.SemanticModelID,'00000000-0000-0000-0000-00000000') AS ReportSemanticModelID,
COALESCE(R.ReportID,'00000000-0000-0000-0000-00000000') AS ReportID,
COALESCE(R.OriginalReportID,'00000000-0000-0000-0000-00000000') AS OriginalReportID,
COALESCE(SM.CapacityID,'00000000-0000-0000-0000-00000000') AS SemanticModelCapacityID,
COALESCE(SM.DomainID,'00000000-0000-0000-0000-00000000') AS SemanticModelDomainID,
COALESCE(SM.WorkspaceID,'00000000-0000-0000-0000-00000000') AS SemanticModelWorkspaceID,
COALESCE(SM.DeploymentPipelineID,'00000000-0000-0000-0000-00000000') AS SemanticModelDeploymentPipelineID,
COALESCE(SM.DeploymentPipelineStageID,'00000000-0000-0000-0000-00000000') AS SemanticModelDeploymentPipelineStageID,
COALESCE(SM.SemanticModelID,'00000000-0000-0000-0000-00000000') AS SemanticModelID
FROM DashboardTiles DT
FULL OUTER JOIN Dashboards D ON DT.DashboardID = D.DashboardID
FULL OUTER JOIN Reports R ON DT.ReportID = R.ReportID
FULL OUTER JOIN SemanticModels SM ON R.SemanticModelID = SM.SemanticModelID
""")

SemanticModelLineage_df.write.mode("overwrite").format("delta").save("Tables/SemanticModelLineage")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 113, Finished, Available)

In [112]:
DatamartLineage_df = spark.sql("""
WITH 
DatamartLineage_Staging AS (
SELECT 
DM.DatamartID,
'00000000-0000-0000-0000-00000000' AS DatasourceID,
'Unknown Datasource' AS LineageGroup,
CONCAT('Datamart: ',DM.DatamartName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM Datamarts DM
LEFT JOIN Workspaces W ON DM.WorkspaceID = W.WorkspaceID
WHERE DM.DatamartID NOT IN (SELECT DISTINCT DatamartID FROM DatamartDatasources)
AND DM.DatamartID NOT IN (SELECT DISTINCT DatamartID FROM DatamartDataflowLinks)
AND DM.DatamartID NOT IN (SELECT DISTINCT DatamartID FROM DatamartDatamartLinks)
AND DM.DatamartID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT 
DMDS.DatamartID,
COALESCE(DMDS.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
'Via Datamart' AS LineageGroup,
CONCAT('Datamart: ',DM.DatamartName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM DatamartDatasources DMDS
LEFT JOIN Datamarts DM ON DMDS.DatamartID = DM.DatamartID
LEFT JOIN Workspaces W ON DM.WorkspaceID = W.WorkspaceID
WHERE DMDS.DatamartID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT 
DMDFL.DatamartID,
COALESCE(DFDS.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
'Via Upstream Dataflow' AS LineageGroup,
CONCAT('Dataflow: ',DF.DataflowName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM DatamartDataflowLinks DMDFL
JOIN DataflowDatasources DFDS ON DMDFL.TargetDataflowID = DFDS.DataflowID
LEFT JOIN Dataflows DF ON DMDFL.TargetDataflowID = DF.DataflowID
LEFT JOIN Workspaces W ON DF.WorkspaceID = W.WorkspaceID
WHERE DMDFL.DatamartID <> '00000000-0000-0000-0000-00000000'
),

DatamartLineage AS (
SELECT 
DMDML.DatamartID,
COALESCE(DMLS.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
'Via Upstream Datamart' AS LineageGroup,
CONCAT('Datamart: ',UDM.DatamartName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM DatamartDatamartLinks DMDML
JOIN DatamartLineage_Staging DMLS ON DMDML.TargetDatamartID = DMLS.DatamartID
LEFT JOIN Datamarts UDM ON DMDML.TargetDatamartID = UDM.DatamartID
LEFT JOIN Workspaces W ON UDM.WorkspaceID = W.WorkspaceID
WHERE DMDML.DatamartID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT * FROM DatamartLineage_Staging
)

SELECT 
DatamartID,
DatasourceID,
LineageGroup,
LineageDetail
FROM DatamartLineage
""")

DatamartLineage_df.write.mode("overwrite").format("delta").save("Tables/DatamartLineage")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 114, Finished, Available)

In [113]:
DataflowLineage_df = spark.sql("""
WITH 
DataflowLineage_Staging AS (
SELECT 
DF.DataflowID,
'00000000-0000-0000-0000-00000000' AS DatasourceID,
'Unknown Datasource' AS LineageGroup,
CONCAT('Dataflow: ',DF.DataflowName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM Dataflows DF
LEFT JOIN Workspaces W ON DF.WorkspaceID = W.WorkspaceID
WHERE DF.DataflowID NOT IN (SELECT DISTINCT DataflowID FROM DataflowDatasources)
AND DF.DataflowID NOT IN (SELECT DISTINCT DataflowID FROM DataflowDataflowLinks)
AND DF.DataflowID NOT IN (SELECT DISTINCT DataflowID FROM DataflowDataflowLinks)
AND DF.DataflowID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT 
DFDS.DataflowID,
COALESCE(DFDS.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
'Via Dataflow' AS LineageGroup,
CONCAT('Dataflow: ',DF.DataflowName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM DataflowDatasources DFDS
LEFT JOIN Dataflows DF ON DFDS.DataflowID = DF.DataflowID
LEFT JOIN Workspaces W ON DF.WorkspaceID = W.WorkspaceID
WHERE DFDS.DataflowID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT 
DFDML.DataflowID,
COALESCE(DMDS.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
'Via Upstream Datamart' AS LineageGroup,
CONCAT('Datamart: ',DM.DatamartName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM DataflowDatamartLinks DFDML
LEFT JOIN DatamartDatasources DMDS ON DFDML.TargetDatamartID = DMDS.DatamartID
LEFT JOIN Datamarts DM ON DFDML.TargetDatamartID = DM.DatamartID
LEFT JOIN Workspaces W ON DM.WorkspaceID = W.WorkspaceID
WHERE DFDML.DataflowID <> '00000000-0000-0000-0000-00000000'
),

DataflowLineage AS (
SELECT 
DFDFL.DataflowID,
COALESCE(DFLS.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
'Via Upstream Dataflow' AS LineageGroup,
CONCAT('Dataflow: ',UDF.DataflowName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM DataflowDataflowLinks DFDFL
JOIN DataflowLineage_Staging DFLS ON DFDFL.TargetDataflowID = DFLS.DataflowID
LEFT JOIN Dataflows UDF ON DFDFL.TargetDataflowID = UDF.DataflowID
LEFT JOIN Workspaces W ON UDF.WorkspaceID = W.WorkspaceID
WHERE DFDFL.DataflowID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT * FROM DataflowLineage_Staging
)

SELECT 
DataflowID,
DatasourceID,
LineageGroup,
LineageDetail 
FROM DataflowLineage
""")

DataflowLineage_df.write.mode("overwrite").format("delta").save("Tables/DataflowLineage")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 115, Finished, Available)

In [114]:
SemanticModelDatasourceLineage_df = spark.sql("""
WITH SemanticModelDatasources_Staging AS (

SELECT 
SM.SemanticModelID,
'00000000-0000-0000-0000-00000000' AS DataflowID,
'00000000-0000-0000-0000-00000000' AS DatamartID,
'00000000-0000-0000-0000-00000000' AS DatasourceID,
'Unknown Datasource' AS LineageGroup,
'Unknown' AS LineageDetail
FROM SemanticModels SM
WHERE SM.SemanticModelID NOT IN (SELECT DISTINCT SemanticModelID FROM SemanticModelDatasources)
AND SM.SemanticModelID NOT IN (SELECT DISTINCT SemanticModelID FROM SemanticModelDataflowLinks)
AND SM.SemanticModelID NOT IN (SELECT DISTINCT SemanticModelID FROM SemanticModelDatamartLinks)
AND SM.SemanticModelID NOT IN (SELECT DISTINCT SemanticModelID FROM SemanticModelSemanticModelLinks)
AND SM.SemanticModelID <> '00000000-0000-0000-0000-00000000'

UNION ALL

SELECT 
SMDS.SemanticModelID,
'00000000-0000-0000-0000-00000000' AS DataflowID,
'00000000-0000-0000-0000-00000000' AS DatamartID,
SMDS.DatasourceID,
'Via Power Query' AS LineageGroup,
'This Semantic Model' AS LineageDetail
FROM SemanticModelDatasources SMDS

UNION ALL

SELECT 
SMDFL.SemanticModelID,
DFL.DataflowID,
'00000000-0000-0000-0000-00000000' AS DatamartID,
DFL.DatasourceID,
DFL.LineageGroup,
DFL.LineageDetail
FROM SemanticModelDataflowLinks SMDFL
JOIN DataflowLineage DFL ON SMDFL.TargetDataflowID = DFL.DataflowID

UNION ALL

SELECT 
SMDML.SemanticModelID,
'00000000-0000-0000-0000-00000000' AS DataflowID,
DMDS.DatamartID,
DMDS.DatasourceID,
DMDS.LineageGroup,
DMDS.LineageDetail
FROM SemanticModelDatamartLinks SMDML
JOIN DatamartLineage DMDS ON SMDML.TargetDatamartID = DMDS.DatamartID
),

SemanticModelDatasources AS (
SELECT 
SMSML.SemanticModelID,
SMDS.DataflowID,
SMDS.DatamartID,
SMDS.DatasourceID,
'Via Composite Model' AS LineageGroup,
CONCAT('Semantic Model: ',SM.SemanticModelName,' | Workspace: ',W.WorkspaceName) AS LineageDetail
FROM SemanticModelSemanticModelLinks SMSML
JOIN SemanticModels SM ON SMSML.TargetSemanticModelID = SM.SemanticModelID
JOIN Workspaces W ON SM.WorkspaceID = W.WorkspaceID
JOIN SemanticModelDatasources_Staging SMDS ON SMSML.TargetSemanticModelID = SMDS.SemanticModelID

UNION ALL

SELECT * FROM SemanticModelDatasources_Staging
)

SELECT * FROM SemanticModelDatasources
""")

SemanticModelDatasourceLineage_df.write.mode("overwrite").format("delta").save("Tables/SemanticModelDatasourceLineage")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 116, Finished, Available)

In [115]:
PowerBILineage_df = spark.sql("""
SELECT 
SML.DashboardTileCapacityID,
SML.DashboardTileDomainID,
SML.DashboardTileWorkspaceID,
SML.DashboardTileDeploymentPipelineID,
SML.DashboardTileDeploymentPipelineStageID,
SML.DashboardTileSemanticModelID,
SML.DashboardTileReportID,
SML.DashboardTileID,
SML.DashboardCapacityID,
SML.DashboardDomainID,
SML.DashboardWorkspaceID,
SML.DashboardDeploymentPipelineID,
SML.DashboardDeploymentPipelineStageID,
SML.DashboardAppID,
SML.DashboardID,
SML.ReportCapacityID,
SML.ReportDomainID,
SML.ReportWorkspaceID,
SML.ReportDeploymentPipelineID,
SML.ReportDeploymentPipelineStageID,
SML.ReportAppID,
SML.ReportSemanticModelID,
SML.ReportID,
SML.OriginalReportID,
SML.SemanticModelCapacityID,
SML.SemanticModelDomainID,
SML.SemanticModelWorkspaceID,
SML.SemanticModelDeploymentPipelineID,
SML.SemanticModelDeploymentPipelineStageID,
SML.SemanticModelID,
COALESCE(DF.CapacityID,'00000000-0000-0000-0000-00000000') AS DataflowCapacityID,
COALESCE(DF.DomainID,'00000000-0000-0000-0000-00000000') AS DataflowDomainID,
COALESCE(DF.WorkspaceID,'00000000-0000-0000-0000-00000000') AS DataflowWorkspaceID,
COALESCE(DF.DeploymentPipelineID,'00000000-0000-0000-0000-00000000') AS DataflowDeploymentPipelineID,
COALESCE(DF.DeploymentPipelineStageID,'00000000-0000-0000-0000-00000000') AS DataflowDeploymentPipelineStageID,
COALESCE(DF.DataflowID,'00000000-0000-0000-0000-00000000') AS DataflowID,
COALESCE(DM.CapacityID,'00000000-0000-0000-0000-00000000') AS DatamartCapacityID,
COALESCE(DM.DomainID,'00000000-0000-0000-0000-00000000') AS DatamartDomainID,
COALESCE(DM.WorkspaceID,'00000000-0000-0000-0000-00000000') AS DatamartWorkspaceID,
COALESCE(DM.DeploymentPipelineID,'00000000-0000-0000-0000-00000000') AS DatamartDeploymentPipelineID,
COALESCE(DM.DeploymentPipelineStageID,'00000000-0000-0000-0000-00000000') AS DatamartDeploymentPipelineStageID,
COALESCE(DM.DatamartID,'00000000-0000-0000-0000-00000000') AS DatamartID,
COALESCE(SMDSL.DatasourceID,'00000000-0000-0000-0000-00000000') AS DatasourceID,
COALESCE(SMDSL.LineageGroup,'Invalid Datasource') AS LineageGroup,
COALESCE(SMDSL.LineageDetail,'Invalid Datasource') AS LineageDetails
FROM SemanticModelLineage SML
LEFT JOIN SemanticModelDatasourceLineage SMDSL ON SML.SemanticModelID = SMDSL.SemanticModelID
FULL OUTER JOIN Dataflows DF ON SMDSL.DataflowID = DF.DataflowID
FULL OUTER JOIN Datamarts DM ON SMDSL.DatamartID = DM.DatamartID
""")

PowerBILineage_df.write.mode("overwrite").format("delta").save("Tables/PowerBILineage")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 117, Finished, Available)

##### Delete the files used in processing to save on storage costs

In [116]:
path = nameOfDatasourcesFileFolder
listoffiles= mssparkutils.fs.ls(path)
if len(listoffiles) > 0:       
    for file in listoffiles:
        mssparkutils.fs.rm(file.path)
print(f"All files in {path} deleted.")

path = nameOfWorkspacesFileFolder
listoffiles= mssparkutils.fs.ls(path)
if len(listoffiles) > 0:       
    for file in listoffiles:
        mssparkutils.fs.rm(file.path)
print(f"All files in {path} deleted.")

path = nameOfTempParquetFileFolder
mssparkutils.fs.rm(path)
mssparkutils.fs.mkdirs(path)
print(f"All files in {path} deleted.")

StatementMeta(, 7f3307a6-cd13-4ee7-af20-9bca0505e69d, 118, Submitted, Running)

##### Update the timestamp in the Control file

In [ ]:
j = {'modifiedSince':modifiedSinceNext}
with open(f'/lakehouse/default/{nameOfControlFileFolder}/Control.json', 'w') as f:
    f.write(json.dumps(j))
    f.close()

StatementMeta(, , , Waiting, )